**Project Check-in 5**

In [ ]:
%pip install --upgrade pip
%pip install scikit-lego
%pip install seaborn
%pip install nbstripout
!nbstripout --install

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import keras
df = pd.read_csv("./dataset.csv")

In [ ]:
# Step 1: Clean Data
# Remove duplicates
df_cleaned = df.drop(columns='Unnamed: 0').drop_duplicates(subset=['track_id','album_name','artists','track_name'])

# Remove columns with every row unique. Also dropping artist and album because it would be too much one-hot encoding
df_cleaned.drop(columns=['track_id', 'track_name', 'artists','album_name'], inplace=True)
df_cleaned.dropna(axis=0,inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

#The columns with object datatype will be categorical
columns = df_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()
df_cleaned = df_cleaned[columns]

y = df_cleaned["popularity"]
y = np.array([float(i) for i in y])
print(y)
scaler = StandardScaler() # Scale the data so that the variances for each feature can be similarly weighted
df_cleaned = scaler.fit_transform(df_cleaned)
df_cleaned = pd.DataFrame(df_cleaned, columns=columns)

In [ ]:
df_cleaned


2. We evaluated our NN based on the mean squared error between predicted and true values for popularity on our validation dataset. We also graphed predicted values vs true values for popularity.

3. We trained our NN using the built-in learning framework for keras, which utilizes batch gradient descent. Learning rate of 0.001 is recommended for Adam optimizer. Adam will adaptively adjust the learning rate based on an exponentially weighted history of the gradients, so we have "momentum" built-in to our learning rate.

In [ ]:
X = df_cleaned.drop(columns="popularity")
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = keras.Sequential([
    keras.layers.Input(shape=(13,)),
    keras.layers.Dense(64, activation='relu'),  # Fully connected layer with 64 units
    keras.layers.Dense(64, activation='relu'),  # Fully connected layer with 64 units
    keras.layers.Dense(1)  # Output layer for 10 classes
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['root_mean_squared_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))
# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Over Time')
plt.legend()
plt.show()

# Generate predictions
predictions = model.predict(X_test).flatten()

# Create a DataFrame to compare predictions with real values
comparison_df = pd.DataFrame({
    'Real Value': y_test,
    'Predicted Value': predictions
})
plt.xlabel("True Value")
plt.ylabel("Predicted Value")
plt.scatter(y_test, predictions)
# Display a sample of the table
print(comparison_df.sample(10))  # Show 10 random samples